<a href="https://colab.research.google.com/github/Hoirotunnisa/UAS-AMS/blob/main/Salinan_dari_Salinan_dari_AMS_Priojek_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import requests
import string

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud
from collections import Counter

In [ ]:
df = pd.read_csv('/content/AMS - Sheet1.csv')
df = df.rename(columns={'text':'full_text'}, inplace=False)
df.sample(10)


,input,likedByAuthor,createTimeISO,uniqueId,full_text,createTime,label
4203,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-14T14:21:22.000Z,bluecode.08,"Gubernurnya kadang pakai pencitraan aja, kl sa...",1741962082,0
3345,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-04T21:29:49.000Z,armalia.purwati,Asliii nih. gw suka leadership nya kang Deddy,1746394189,0
4164,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-10T13:29:21.000Z,renn.2gd,itu diancurin karna gaada surat izin nya,1741613361,0
497,https://vt.tiktok.com/ZSBCpDAL7/,False,2025-03-06T21:24:25.000Z,yerikamies,"sayang sih, berarti uang nya pake kas negara y...",1741296265,2
2518,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-23T10:41:15.000Z,aliboge11,Bpk aing,1747996875,1
4156,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-13T14:59:22.000Z,davinjonathan1402,sambungin lagu gua\ndi negri nongeng ada si oki,1741877962,2
3692,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-13T06:36:27.000Z,jerrypagarbuana96,bpk DEDI Mulyadi well pokonamah ....,1747118187,1
1204,https://vt.tiktok.com/ZSBCpDAL7/,False,2025-03-06T10:28:34.000Z,chiki.chiki328,dulu yg sahkan IMB siapa? tolong tangkap itu p...,1741256914,0
3960,https://www.tiktok.com/@afianrazi/video/747903...,False,2025-03-09T17:50:25.000Z,khobul25,demi warganya,1741542625,0
2910,https://vt.tiktok.com/ZSBCtsQpF/,False,2025-05-10T15:53:39.000Z,arul.sahroni,gaskeun,1746892419,0


# PREPROCESSING

In [ ]:
# Mengecek data yang null/kosong
print(df.isnull().sum())
df = df.dropna()

input            0
likedByAuthor    0
createTimeISO    0
uniqueId         0
full_text        0
createTime       0
label            0
dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5139 entries, 0 to 5138
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   input          5139 non-null   object
 1   likedByAuthor  5139 non-null   bool  
 2   createTimeISO  5139 non-null   object
 3   uniqueId       5139 non-null   object
 4   full_text      5139 non-null   object
 5   createTime     5139 non-null   int64 
 6   label          5139 non-null   int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 246.0+ KB


In [ ]:
# Mengubah tipe data
df['label'] = df['label'].astype('int')
df['label'] = df['label'].astype('category')

In [ ]:
# Mangambil kolom 'full_text' untuk text cleaning
df_text = df[['full_text']]
df_text

,full_text
0,"lagian malah bangun kayagitu, udah bagus bagus..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...
2,ini nih yg menghilangkan area resapan air. gun...
3,Penyebab banjir bukan tempat wisata #hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua..."
...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...
5136,memang kalau hbs sakit ibarat e sembuh total.....
5137,sedih Sekali liatin bangunan Nya di hancur kan...


In [ ]:
import re

def remove_URL(text): # Menghapus URL
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text): # Menghapus tag HTML
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text): # Menghapus emoji
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
                            "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers (text): # Menghapus angka
    text = re.sub(r'\d+', '', text)
    return text

def remove_duplspaces (text): # Menghapus spasi berlebih
    text = re.sub(r'\s+', ' ', text)
    return text

def remove_symbols (text): # Menghapus simbol
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

In [ ]:
df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_emoji(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_symbols(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_numbers(x))
df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_duplspaces(x))
df_text

/tmp/ipython-input-9-841270229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['full_text'].apply(lambda x: remove_URL(x))
/tmp/ipython-input-9-841270229.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cleaning'] = df_text['cleaning'].apply(lambda x: remove_html(x))
/tmp/ipython-input-9-841270229.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,full_text,cleaning
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...
...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...


In [ ]:
# Menghapus duplikasi
df_text = df_text.drop_duplicates(subset='cleaning', keep='first')
df_text

,full_text,cleaning
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...
...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...


In [ ]:
# Case Folding
def case_folding(text): # Mengubah huruf yang masih uppercase menjadi lowercase
  if isinstance(text, str):
    lowercase_text = text.lower()
    return lowercase_text
  else:
    return text

df_text['case_folding'] = df_text['cleaning'].apply(case_folding)
df_text

/tmp/ipython-input-11-1189230327.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['case_folding'] = df_text['cleaning'].apply(case_folding)


,full_text,cleaning,case_folding
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...
...,...,...,...
5134,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...
5135,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...
5136,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...
5137,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...


# TOKENISASI

In [ ]:
def tokenize_with_nltk(text):
    """
    Tokenisasi teks menggunakan NLTK, mengembalikan list kata saja.
    :param text: String teks input
    :return: List token kata
    """
    return word_tokenize(text)

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
df_text['token'] = df_text['token'].apply(set)
df_text['token'] = df_text['token'].apply(list)
df_text = df_text.reset_index(drop=True)

/tmp/ipython-input-13-2255154632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['case_folding'].apply(tokenize_with_nltk)
/tmp/ipython-input-13-2255154632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['token'] = df_text['token'].apply(set)
/tmp/ipython-input-13-2255154632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
df_text

,full_text,cleaning,case_folding,token
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[malah, lagian, bagus, kayagitu, teh, bangun, ..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[dampak, disitu, pembongkaranmana, ini, piknik..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[yg, hujan, ini, dialirkan, ke, gunung, air, d..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[tempat, penyebab, wisata, bukan, banjir, hibisc]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[yg, dijamin, sy, terjadi, keluar, teh, lmyan,..."
...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[dari, petugas, dan, perkebunan, pikiran, utk,..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[yg, hiburan, hujan, lahan, dpt, butuh, utk, t..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[pejabat, ibarat, penakora, sakit, sembuh, dik..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[mata, siap, kan, nya, dgn, sedih, baru, banji..."


# SLANG

In [ ]:
!wget -N -P data/ https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}

--2025-07-16 11:59:12--  https://docs.google.com/spreadsheets/d/%7Bsheet_id%7D/gviz/tq?tqx=out:csv
Resolving docs.google.com (docs.google.com)... 173.194.216.113, 173.194.216.101, 173.194.216.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.216.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-16 11:59:12 ERROR 404: Not Found.



In [ ]:
import requests
import os

sheet_id = "18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4"
sheet_name = "Sheet1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
file_path = 'data/slang.txt'

# Create 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

try:
    response = requests.get(url)
    response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

    with open(file_path, 'wb') as f:
        f.write(response.content)

    print(f"File downloaded and saved as '{file_path}'")

except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

File downloaded and saved as 'data/slang.txt'


In [ ]:
def replace_slang_in_text(file_path, tokens):
    """
    Load the slang dictionary from a file and replace slang words in a list of tokens.
    The file should have lines formatted as "slang:replacement".
    """
    slang_dict = {}
    # Load slang dictionary
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ':' in line:
                slang, replacement = line.strip().split(':', 1)
                slang_dict[slang.strip()] = replacement.strip()

    # Replace slang words in tokens
    return [slang_dict.get(token, token) for token in tokens]

In [ ]:
# Menggunakan file pertama ('slang.txt')
df_text["slang1"] = df_text["token"].apply(lambda tokens: replace_slang_in_text('data/slang.txt', tokens))

In [ ]:
df_text

,full_text,cleaning,case_folding,token,slang1
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[malah, lagian, bagus, kayagitu, teh, bangun, ...","[malah, lagian, bagus, kayagitu, teh, bangun, ..."
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, ini, piknik..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, ini, dialirkan, ke, gunung, air, d..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[tempat, penyebab, wisata, bukan, banjir, hibisc]","[tempat, penyebab, wisata, bukan, banjir, hibisc]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, terjadi, keluar, teh, lmyan,..."
...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[dari, petugas, dan, perkebunan, pikiran, utk,...","[dari, petugas, dan, perkebunan, pikiran, utk,..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[pejabat, ibarat, penakora, sakit, sembuh, dik...","[pejabat, ibarat, penakora, sakit, sembuh, dik..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[mata, siap, kan, nya, dgn, sedih, baru, banji...","[mata, siap, kan, nya, dgn, sedih, baru, banji..."


# STOPWORD

In [ ]:
!wget -O data/Stopword.csv "https://docs.google.com/spreadsheets/d/18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4/gviz/tq?tqx=out:csv&sheet=Stopword"


# Membaca file stopwords_danantara.txt SEKALI di awal
with open('data/Stopword.csv', "r", encoding="utf-8", errors='replace') as file:
    manual_stop = file.readlines()

# Memproses dan menghapus duplikat
manual_stop = [word.strip().lower() for word in manual_stop]
manual_stop = list(set(manual_stop)) # Mengubah ke set untuk menghapus duplikat, lalu kembali ke list

--2025-07-16 11:59:56--  https://docs.google.com/spreadsheets/d/18MdbZse59vpAfsyCDfZSee_iojVb1etbiyTJ5AA0Lj4/gviz/tq?tqx=out:csv&sheet=Stopword
Resolving docs.google.com (docs.google.com)... 173.194.217.102, 173.194.217.101, 173.194.217.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.217.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data/Stopword.csv’

data/Stopword.csv       [ <=>                ] 882.01K  --.-KB/s    in 0.1s    

2025-07-16 11:59:57 (7.07 MB/s) - ‘data/Stopword.csv’ saved [903175]



In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
id_stop = stopwords.words('indonesian')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Fungsi yang menggunakan daftar stopwords yang sudah dimuat
def remove_stopwords(tokens, stopwords_list):
    """
    Remove stopwords from a list of tokens using a predefined stopword list.
    """
    return [token for token in tokens if token not in stopwords_list]

In [ ]:
# Terapkan penghapusan stopwords ke kolom 'slang1'
df_text['no_stopwords'] = df_text['slang1'].apply(lambda tokens: remove_stopwords(tokens, id_stop))

# Terapkan penghapusan stopwords ke kolom 'cleaned_text'
df_text['m_no_stopwords'] = df_text['no_stopwords'].apply(lambda tokens: remove_stopwords(tokens, manual_stop))

In [ ]:
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[malah, lagian, bagus, kayagitu, teh, bangun, ...","[malah, lagian, bagus, kayagitu, teh, bangun, ...","[bagus, kayagitu, teh, bangun, udah, kebun]","[bagus, kayagitu, teh, bangun, udah, kebun]"
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, piknik, teh...","[dampak, disitu, pembongkaranmana, piknik, teh..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, dialirkan, gunung, air, diciptakan...","[yg, hujan, dialirkan, gunung, air, diciptakan..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[tempat, penyebab, wisata, bukan, banjir, hibisc]","[tempat, penyebab, wisata, bukan, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ..."
...,...,...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[dari, petugas, dan, perkebunan, pikiran, utk,...","[dari, petugas, dan, perkebunan, pikiran, utk,...","[petugas, perkebunan, pikiran, utk, mencegah, ...","[petugas, perkebunan, pikiran, utk, mencegah, ..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[pejabat, ibarat, penakora, sakit, sembuh, dik...","[pejabat, ibarat, penakora, sakit, sembuh, dik...","[pejabat, penakora, sakit, sembuh, dikitibarat...","[pejabat, penakora, sakit, sembuh, dikitibarat..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[mata, siap, kan, nya, dgn, sedih, baru, banji...","[mata, siap, kan, nya, dgn, sedih, baru, banji...","[mata, nya, dgn, sedih, banjir, sekejap, kaita...","[mata, nya, dgn, sedih, banjir, sekejap, kaita..."


# LEMMAZITATION

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming menggunakan Sastrawi untuk daftar token
def stem_tokens(tokens):
    """
    Stem each token in a list using Sastrawi stemmer.
    :param tokens: List of tokens to be stemmed.
    :return: List of stemmed tokens.
    """
    return [stemmer.stem(token) for token in tokens]

In [ ]:
df_text['lemma_text'] = df_text['m_no_stopwords'].apply(stem_tokens)
df_text

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[malah, lagian, bagus, kayagitu, teh, bangun, ...","[malah, lagian, bagus, kayagitu, teh, bangun, ...","[bagus, kayagitu, teh, bangun, udah, kebun]","[bagus, kayagitu, teh, bangun, udah, kebun]","[bagus, kayagitu, teh, bangun, udah, kebun]"
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, piknik, teh...","[dampak, disitu, pembongkaranmana, piknik, teh...","[dampak, situ, pembongkaranmana, piknik, teh, ..."
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, dialirkan, gunung, air, diciptakan...","[yg, hujan, dialirkan, gunung, air, diciptakan...","[yg, hujan, alir, gunung, air, cipta, area, ni..."
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[tempat, penyebab, wisata, bukan, banjir, hibisc]","[tempat, penyebab, wisata, bukan, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]","[sebab, wisata, banjir, hibisc]"
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ...","[yg, jamin, sy, teh, lmyan, dr, pohon, air, li..."
...,...,...,...,...,...,...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk ada pikiran dari petugas perkebunan it...,"[dari, petugas, dan, perkebunan, pikiran, utk,...","[dari, petugas, dan, perkebunan, pikiran, utk,...","[petugas, perkebunan, pikiran, utk, mencegah, ...","[petugas, perkebunan, pikiran, utk, mencegah, ...","[tugas, kebun, pikir, utk, cegah, banjir, tdk,..."
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,Hujan butuh hutan Hutanperlu lahan lahanhabis ...,hujan butuh hutan hutanperlu lahan lahanhabis ...,"[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hiburan, hujan, lahan, dpt, butuh, utk, t...","[yg, hibur, hujan, lahan, dpt, butuh, utk, sim..."
4921,memang kalau hbs sakit ibarat e sembuh total.....,memang kalau hbs sakit ibarat e sembuh totallh...,memang kalau hbs sakit ibarat e sembuh totallh...,"[pejabat, ibarat, penakora, sakit, sembuh, dik...","[pejabat, ibarat, penakora, sakit, sembuh, dik...","[pejabat, penakora, sakit, sembuh, dikitibarat...","[pejabat, penakora, sakit, sembuh, dikitibarat...","[jabat, penakora, sakit, sembuh, dikitibarat, ..."
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih Sekali liatin bangunan Nya di hancur kan...,sedih sekali liatin bangunan nya di hancur kan...,"[mata, siap, kan, nya, dgn, sedih, baru, banji...","[mata, siap, kan, nya, dgn, sedih, baru, banji...","[mata, nya, dgn, sedih, banjir, sekejap, kaita...","[mata, nya, dgn, sedih, banjir, sekejap, kaita...","[mata, nya, dgn, sedih, banjir, kejap, kait, h..."


In [ ]:
a = df_text['lemma_text'].to_list()
b = [' '.join(a[i]) for i in range(len(a))]
df_text['cleaned_text'] = b
df_text.head()

,full_text,cleaning,case_folding,token,slang1,no_stopwords,m_no_stopwords,lemma_text,cleaned_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",lagian malah bangun kayagitu udah bagus bagus ...,lagian malah bangun kayagitu udah bagus bagus ...,"[malah, lagian, bagus, kayagitu, teh, bangun, ...","[malah, lagian, bagus, kayagitu, teh, bangun, ...","[bagus, kayagitu, teh, bangun, udah, kebun]","[bagus, kayagitu, teh, bangun, udah, kebun]","[bagus, kayagitu, teh, bangun, udah, kebun]",bagus kayagitu teh bangun udah kebun
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,udh bener puncak itu kebon teh bukaaaaan tmpt ...,"[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, ini, piknik...","[dampak, disitu, pembongkaranmana, piknik, teh...","[dampak, disitu, pembongkaranmana, piknik, teh...","[dampak, situ, pembongkaranmana, piknik, teh, ...",dampak situ pembongkaranmana piknik teh warpat...
2,ini nih yg menghilangkan area resapan air. gun...,ini nih yg menghilangkan area resapan air gunu...,ini nih yg menghilangkan area resapan air gunu...,"[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, ini, dialirkan, ke, gunung, air, d...","[yg, hujan, dialirkan, gunung, air, diciptakan...","[yg, hujan, dialirkan, gunung, air, diciptakan...","[yg, hujan, alir, gunung, air, cipta, area, ni...",yg hujan alir gunung air cipta area nih tangka...
3,Penyebab banjir bukan tempat wisata #hibisc,Penyebab banjir bukan tempat wisata hibisc,penyebab banjir bukan tempat wisata hibisc,"[tempat, penyebab, wisata, bukan, banjir, hibisc]","[tempat, penyebab, wisata, bukan, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]","[penyebab, wisata, banjir, hibisc]","[sebab, wisata, banjir, hibisc]",sebab wisata banjir hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana hari sebelum puasa liat air keluar d...,sy kesana hari sebelum puasa liat air keluar d...,"[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, terjadi, keluar, teh, lmyan,...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ...","[yg, dijamin, sy, teh, lmyan, dr, pohon, air, ...","[yg, jamin, sy, teh, lmyan, dr, pohon, air, li...",yg jamin sy teh lmyan dr pohon air liat wahana...


In [ ]:
df = df[df['full_text'].isin(df_text['full_text'])]
df = df.drop_duplicates(subset='full_text', keep='first')

In [ ]:
df = df.reset_index(drop=True)

# Memisahkan kolom tanggal dan jam

In [ ]:
# Pastikan kolom 'createTimeISO' adalah datetime
df['createTimeISO'] = pd.to_datetime(df['createTimeISO'])

# Memisahkan kolom 'createTimeISO' menjadi 'date' dan 'time'
# date akan tetap berupa objek datetime.date atau timestamp terlebih dahulu
df['date'] = df['createTimeISO'].dt.date
df['time'] = df['createTimeISO'].dt.time

# Mengubah format kolom 'date' menjadi string 'dd/mm/yyyy'
# Konversi ke datetime untuk menggunakan strftime, lalu ke string
df['date'] = pd.to_datetime(df['date']).dt.strftime('%d/%m/%Y')

In [ ]:
df_cleaned = df[['label', 'date', 'time', 'uniqueId', 'input', 'likedByAuthor', 'full_text']]
df_cleaned = pd.concat([df_cleaned, df_text['cleaned_text']], axis=1)
df_cleaned.sample(10)

,label,date,time,uniqueId,input,likedByAuthor,full_text,cleaned_text
1789,2,06/03/2025,08:45:15,user67984636617275,https://vt.tiktok.com/ZSBCpDAL7/,False,setuju di bongkar semulah,tuju bongkar semu
4915,0,07/03/2025,23:28:45,user42283837022,https://www.tiktok.com/@afianrazi/video/747903...,False,sayang banget bangauna mewah di ancurin sayah ...,rumah ancurin mewah susahbanget sayang beginih...
3695,1,11/05/2025,13:01:03,_reyhann86,https://vt.tiktok.com/ZSBCtsQpF/,False,"jangan terllalu berlebihan pak,,",terllalu
952,0,07/03/2025,15:02:40,nrmadinah_376,https://vt.tiktok.com/ZSBCpDAL7/,False,demi apapun puncak tuh gabutuh ini. butuhnya k...,apa bandung tuh butuh mah kek segar mending ko...
3457,2,19/05/2025,01:31:04,sioyen922,https://vt.tiktok.com/ZSBCtsQpF/,False,"keren rugi40M , tp anak cucu selmat di masa yg...",yg selmat dtg anak nya rugim aing berfikir ker...
3450,2,18/05/2025,15:17:18,rocket_queengnr,https://vt.tiktok.com/ZSBCtsQpF/,False,kawasan hijau kok disemen seperti itu,kawasan hijau semen
2346,2,30/03/2025,08:52:49,berkahpengusahamuda,https://vt.tiktok.com/ZSBCtsQpF/,False,"Mantaap bener ini Gubernur Jabar,..TOP Markoto...",jabartop gubernur ide mantaap markotop nyali b...
2111,2,06/03/2025,09:17:34,userkablde3399725824254,https://vt.tiktok.com/ZSBCpDAL7/,False,bassiiiiiii\nudah tau pembabadan daerah penghi...,yg kenyang tau ttd serap tolak air d babad gar...
1646,2,06/03/2025,11:06:12,mauliarahayu97,https://vt.tiktok.com/ZSBCpDAL7/,False,gila gimana ga banjir seluas itu hutan dibabad,luas banjir ga hutan gila babad gimana
4297,1,08/03/2025,23:32:30,chanelsanukri,https://www.tiktok.com/@afianrazi/video/747903...,False,lahan parkir luas bener,bener luas parkir lahan


# Menambahkan kolom judul video

In [ ]:
video_link = []
for video_id in df_cleaned['input'].unique():
    link = f'https://vt.tiktok.com/watch?v={input}'
    video_link.append(link)

video_link

['https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x789f981624d0>>',
 'https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x789f981624d0>>',
 'https://vt.tiktok.com/watch?v=<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x789f981624d0>>']

# Membuat dataframe untuk analisis bertopik

In [ ]:
def stopword_berd(text):
    indonesian_stopwords = stopwords.words('indonesian')
    text = " ".join([word for word in text.split() if word not in indonesian_stopwords])
    return text

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df_brt = df[['full_text']]

In [ ]:
df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_symbols(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_numbers(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_duplspaces(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: case_folding(x))
df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: stopword_berd(x))
df_brt

/tmp/ipython-input-39-3486542895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['full_text'].apply(lambda x: remove_URL(x))
/tmp/ipython-input-39-3486542895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_brt['clean_text'] = df_brt['clean_text'].apply(lambda x: remove_html(x))
/tmp/ipython-input-39-3486542895.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

,full_text,clean_text
0,"lagian malah bangun kayagitu, udah bagus bagus...",bangun kayagitu udah bagus bagus kebun teh
1,udh bener puncak itu kebon teh .bukaaaaan tmpt...,udh bener puncak kebon teh bukaaaaan tmpt pikn...
2,ini nih yg menghilangkan area resapan air. gun...,nih yg menghilangkan area resapan air gunung d...
3,Penyebab banjir bukan tempat wisata #hibisc,penyebab banjir wisata hibisc
4,"sy kesana 4 hari sebelum puasa, liat air kelua...",sy kesana puasa liat air dr akar pohon teh yg ...
...,...,...
4919,mmg TDK ada pikiran dari Petugas Perkebunan it...,mmg tdk pikiran petugas perkebunan utk mencega...
4920,Hujan ...butuh hutan\nHutan...perlu lahan\nlah...,hujan butuh hutan hutanperlu lahan lahanhabis ...
4921,memang kalau hbs sakit ibarat e sembuh total.....,hbs sakit e sembuh totallha enakyo dikitibarat...
4922,sedih Sekali liatin bangunan Nya di hancur kan...,sedih liatin bangunan nya hancur begituberapa ...


# Menyimpan dataset

In [ ]:
df_cleaned.to_excel('/content/df_cleaned.xlsx', index=False)
df_brt.to_excel('/content/df_brt.xlsx', index=False)